## Data files used below are sourced from
https://cricsheet.org/downloads/india.zip
    
manually filtered for IND-AUS matches

In [176]:
from ruamel.yaml import YAML
import pandas as pd
import numpy as np
import os, glob
from pathlib import Path

# Team data sourced from https://www.cricbuzz.com/cricket-series/2773/australia-tour-of-india-2019/squads
india_team = ['R Sharma', 'S Dhawan', 'V Kohli', 'AT Rayudu', 'KL Rahul', 'R Pant', 'MS Dhoni', 'V Shankar', 'Kuldeep Yadav',
           'S Kaul', 'B Kumar', 'Mohammed Shami', 'JJ Bumrah', 'Y Chahal', 'RA Jadeja' ]

aus_team = ['AJ Finch', 'A Carey', 'UT Khawaja', 'SE Marsh', 'DA Short', 'PSP Handscomb', 'M Stoinis', 'GJ Maxwell', 'KW Richardson',
        'NM Coulter-Nile', 'J Behrendorff', 'NM Lyon', 'A Zampa']

## Function get_info_from_delivery() extracts desired data from a delivery record

In [177]:
def get_info_from_delivery(rows, yamlfile):
    tmp_list = []
    for row in rows:
        for ball_counter, val in row.items():
            tmp_dict = {}
            tmp_dict['filename'] = yamlfile
            tmp_dict['batsman']=val['batsman']
            tmp_dict['bowler']=val['bowler']
            tmp_dict['runs']=val['runs']['batsman']
            
            for key in val.keys():
                if (key == 'wicket'):
                    tmp_dict['wicket_type'] = val['wicket']['kind']
                    for w_key in val['wicket'].keys():
                        if (w_key == 'fielders'):
                            tmp_dict['fielder'] = val['wicket']['fielders'][0]
                    tmp_dict['player_out']=val['wicket']['player_out']
            tmp_list.append(tmp_dict)
    return(tmp_list)

## Parse all YAML files and build custom dictionary with fields of interest to be used as initializer for dataframe. Filename is included in dictionary for traceability.

### Match data from 2008 and later have been used. Other data is discarded.

In [178]:
player_df_init_list = []
winner_df_init_list = []
unused_files = []

#p=Path(r'/home/ec2-user/UpGrad_IPL/Datafiles/IND-AUS')
p=Path(r'./Datafiles/IND-AUS')

filelist = list(p.glob( "**/*.yaml" ))

yaml=YAML(typ='safe')

for yamlfile in filelist:
    datafile = open(yamlfile, 'r')
    mydict=yaml.load(datafile)
    if ( mydict['info']['dates'][0].year > 2008 ):
        if ( mydict['info']['gender'] == 'male' ):
            if (len(mydict['innings']) > 0):
                if ( 'result' not in mydict['info']['outcome'].keys() ):
                    tmp_dict = {}
                    tmp_dict['country'] = mydict['info']['outcome']['winner']
                    winner_df_init_list.append(tmp_dict)
                for inn in mydict['innings']:
                    idx = 0
                    for inn_key in ['1st innings', '2nd innings']:
                        if (inn_key in inn.keys()):
                            print('Fetching data from: ', str( yamlfile ) )
                            player_df_init_list += get_info_from_delivery( inn[inn_key]['deliveries'], str( yamlfile.name ) )
                        idx += 1
            else:
                unused_files.append( str(yamlfile) + ': No innings data' )
        else:
            unused_files.append( str(yamlfile) + ': Female team' )
    else:
        unused_files.append( str(yamlfile) + ': Invalid year: ' + str( mydict['info']['dates'][0].year ) )

print('\n\nFollowing files were not used\n')
print(*unused_files, sep='\n')

Fetching data from:  Datafiles\IND-AUS\1062573.yaml
Fetching data from:  Datafiles\IND-AUS\1062573.yaml
Fetching data from:  Datafiles\IND-AUS\1062574.yaml
Fetching data from:  Datafiles\IND-AUS\1062574.yaml
Fetching data from:  Datafiles\IND-AUS\1062575.yaml
Fetching data from:  Datafiles\IND-AUS\1062575.yaml
Fetching data from:  Datafiles\IND-AUS\1062576.yaml
Fetching data from:  Datafiles\IND-AUS\1062576.yaml
Fetching data from:  Datafiles\IND-AUS\412691.yaml
Fetching data from:  Datafiles\IND-AUS\412691.yaml
Fetching data from:  Datafiles\IND-AUS\415281.yaml
Fetching data from:  Datafiles\IND-AUS\416236.yaml
Fetching data from:  Datafiles\IND-AUS\416236.yaml
Fetching data from:  Datafiles\IND-AUS\416237.yaml
Fetching data from:  Datafiles\IND-AUS\416237.yaml
Fetching data from:  Datafiles\IND-AUS\416238.yaml
Fetching data from:  Datafiles\IND-AUS\416238.yaml
Fetching data from:  Datafiles\IND-AUS\416239.yaml
Fetching data from:  Datafiles\IND-AUS\416239.yaml
Fetching data from:  Da

## Replace '( sub )' in player names. In data files, it has been used to indicate substitute.

In [179]:
tmp_df = pd.DataFrame(player_df_init_list)
tmp_df['fielder'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)
tmp_df['batsman'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)
tmp_df['bowler'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)
tmp_df['player_out'].replace(to_replace='([\a\.\ ]*)( \(sub\))', value=r'\1', regex=True, inplace=True)

## Check data frame looks OK

In [180]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44440 entries, 0 to 44439
Data columns (total 7 columns):
batsman        44440 non-null object
bowler         44440 non-null object
fielder        593 non-null object
filename       44440 non-null object
player_out     842 non-null object
runs           44440 non-null int64
wicket_type    842 non-null object
dtypes: int64(1), object(6)
memory usage: 2.4+ MB


## Create run rate dataframe for batsmen

In [181]:
batsman_df = tmp_df.groupby(['batsman', 'filename']).sum().sort_values(by=['batsman', 'filename']).reset_index() \
    .groupby(['batsman'])['runs'].agg([np.mean, np.size, np.sum]).reset_index()\
    .sort_values(by=['mean', 'size'], ascending=False)\
    .rename(columns={'sum':'score','size':'match_count', 'mean':'average_runs_per_match'})

## Create wicket taker data frame for bowlers

In [182]:
bowler_df = tmp_df[ ~tmp_df['player_out'].isnull() & tmp_df['fielder'].isnull() ].groupby(['bowler', 'filename'])\
    ['batsman'].agg(np.size).reset_index()\
    .groupby(['bowler'])['batsman'].agg([ np.mean, np.size, np.sum ]).reset_index()\
    .sort_values(by=['mean', 'size'], ascending=False).rename(columns={'size':'match_count', 'sum':'wickets', 'mean':'avg_wkt_per_match'})

## Create wicket taker data frame for fielders

In [183]:
fielder_df = tmp_df[ ~(tmp_df['player_out'].isnull() & tmp_df['fielder'].isnull()) ].groupby(['fielder', 'filename'])\
    ['batsman'].agg(np.size).reset_index()\
    .groupby(['fielder'])['batsman'].agg([ np.mean, np.size, np.sum ]).reset_index()\
    .sort_values(by=['mean', 'sum'], ascending=False).rename(columns={'size':'match_count', 'sum':'wickets', 'mean':'avg_wkt_per_match'})

## Check highest average score for batsmen in 2019 IND/AUS teams where matches played by batsman is more than average numbers of matches played. This ensures consistency in performance is accounted for. 

In [184]:
batsman_df[ batsman_df['batsman'].isin( india_team + aus_team ) &\
           ( batsman_df['match_count'] > batsman_df['match_count'].mean() ) ].head()

,batsman,average_runs_per_match,match_count,score
131,V Kohli,50.354167,48,2417
5,AJ Finch,45.894737,19,872
111,S Dhawan,39.653846,26,1031
81,MS Dhoni,34.111111,45,1535
40,GJ Maxwell,31.090909,22,684


## Check highest average score for bowlers in 2019 IND/AUS teams where matches played by bowler is more than average numbers of matches played. This ensures consistency in performance is accounted for. 

In [185]:
bowler_df[ bowler_df['bowler'].isin(aus_team + india_team) &\
           ( bowler_df['match_count'] > bowler_df['match_count'].mean() ) ].head()

,bowler,avg_wkt_per_match,match_count,wickets
31,NM Lyon,2.000000,7,14
40,RA Jadeja,1.333333,18,24
29,Mohammed Shami,1.333333,6,8
17,JJ Bumrah,1.250000,4,5


## Check highest average score for fielders in 2019 IND/AUS teams where matches played by fielder is more than average numbers of matches played. This ensures consistency in performance is accounted for. 

In [186]:
fielder_df[ fielder_df['fielder'].isin(aus_team + india_team) &\
           ( fielder_df['match_count'] > fielder_df['match_count'].mean() ) ].head()

,fielder,avg_wkt_per_match,match_count,wickets
52,MS Dhoni,2.125000,32,68
23,GJ Maxwell,1.500000,10,15
89,V Kohli,1.320000,25,33
74,S Dhawan,1.181818,11,13
69,RA Jadeja,1.117647,17,19


## Create dataframe for match winners

In [187]:
match_df = pd.DataFrame(winner_df_init_list)

### Compare number of wins by both countries

In [188]:
match_df.groupby('country').size().reset_index()

,country,0
0,Australia,25
1,India,27
